# Excel Extract
- This notebook is a template for to extract information via SQL queries using SQLITE and Pandas
- The goal is to query the database and create an excel table with the required information that is ready to be sent to the stakeholders
- In short, we access the database, create a Pandas DataFrame and do the necessary Data Wrangling. Finally, export and format the excel file.

In [27]:
import pandas as pd
import numpy as np
import sqlite3
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.worksheet.table import Table, TableStyleInfo
from datetime import datetime, timedelta

In [24]:
#Functions

def auto_fit_columns(sheet):
    for col in sheet.column:
        max_length = 0
        column = col[0].column_letter

        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass

        adjusted_width = (max_length + 4)
        sheet.column_dimensions[column].width = adjusted_width

def table_formatting(excel_file, sheet, table_name):
    wb = load_workbook(excel_file)
    ws = wb[sheet]

    start_row = 1
    max_row = ws.max_row
    start_col = 1
    max_col = ws.max_column

    #Turns data into excel table
    table = Table(displayName=table_name, ref=ws.dimensions)
    style = TableStyleInfo(name="TableStyleMedium4",
    showRowStripes=True)
    table.tableStyleInfo = style

    #Header Formatting Prep
    header_font = Font(color="FFFFFF", bold=True)
    no_border = Border(left=Side(border_style=None),
                       right=Side(border_style=None),
                       top=Side(border_style=None),
                       bottom=Side(border_style=None)
                    )
    
    #Formatting Header
    for col in range(1, max_col + 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.alignment = Alignment(horizontal="left")
        cell.border = no_border

    ws.add_table(table)
    auto_fit_columns(ws)
    wb.save(excel_file)

In [25]:
con = sqlite3.connect("sqlite file path")

query = """
SELECT
p.code AS "Code",
a.name AS "Status",
b.bcode AS "B Code",

CASE
    WHEN b.bcode IN ("z", "x")
    THEN "TRUE"
    ELSE "FALSE"
END AS "Selected Entity"

FROM p* AS p

LEFT JOIN b* AS b
ON p.bcode = b.bcode

LEFT JOIN activity AS a
ON p.status_code = a.code

WHERE "Selected Entity" = "TRUE"
AND a.name IN ("ACTIVE", "NOT ACTIVE")
"""

In [26]:
# Get the previous month name to use on the file name
today = datetime.now()
first_day_of_the_current_month = today.replace(day=1)
last_day_of_the_previous_month = first_day_of_the_current_month - timedelta(days=1)

previous_month_name = last_day_of_the_previous_month.strftime("%B")

In [30]:
try:
    extract_file = pd.read_sql_query(query,con)
    extract_file["Code"] = extract_file["Code"].astype(int)
    path = f"path/file_name_({previous_month_name}).xlsx"
    extract_file.to_excel(path, index=False)
    table_formatting(path, "Sheet1", "extract")
    print("Generated Successfully")
except:
    print("Unable to Generate File")


Unable to Generate File
